In [ ]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 4.2 MB/s 
     |████████████████████████████████| 3.3 MB 35.3 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 636 kB 44.1 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import argparse
from dataclasses import dataclass
import json
import logging
import os
import random
import sys
import time
import warnings
import numpy as np
import scipy.stats as st
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

import transformers
#import wandb
from tqdm.auto import tqdm, trange


logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s: %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [ ]:
@dataclass
class CustomArguments(transformers.TrainingArguments):
    sample_train: int = 0
    sample_eval: int = 0
    num_choices: int = 0
    model_name_or_path: str = "asdf"  # this is no longer a TrainingArgument attribute
        
    # python dataclasses cannot have positional attributes in subclass,
    # so give all attributes defaults and then make sure they are changed
    def __post_init__(self):
        if not (self.sample_train * self.sample_eval * self.num_choices) or \
               self.model_name_or_path == "asdf":  # make sure none are still default value
            raise TypeError("__init__ missing required argument(s)")

def get_args():
    """ Set hyperparameters """
    args = CustomArguments(
        output_dir="checkpoint",
        model_name_or_path="gpt2",
        overwrite_output_dir=True,
        do_train=False,  # Zero shot
        do_eval=True,
        per_device_eval_batch_size=2,
        learning_rate=1e-5,  # Should not matter because not training
        weight_decay=0.1,
        save_total_limit=2,
        seed=123,
        sample_train=200,
        sample_eval=-1,
        num_choices=3,
    )
    
    return args

In [ ]:
def get_data(file_path, sample, num_choices):
    data_file = open(file_path, "r")
    logger.info("Reading QA instances from jsonl dataset at: %s", file_path)
    item_jsons = []
    item_ids = []
    questions = []
    choice_lists = []
    answer_ids = []
    for line in data_file:
        item_jsons.append(json.loads(line.strip()))

    if sample != -1:
        item_jsons = random.sample(item_jsons, sample)
        logger.info("Sampling %d examples", sample)

    for item_json in tqdm(item_jsons,total=len(item_jsons)):
        item_id = item_json["id"]

        question_text = item_json["question"]["stem"]

        choice_label_to_id = {}
        choice_text_list = []
        choice_context_list = []
        choice_label_list = []
        choice_annotations_list = []

        any_correct = False
        choice_id_correction = 0

        for choice_id, choice_item in enumerate(item_json["question"]["choices"]):
            choice_label = choice_item["label"]
            choice_label_to_id[choice_label] = choice_id - choice_id_correction
            choice_text = choice_item["text"]

            choice_text_list.append(choice_text)
            choice_label_list.append(choice_label)

            if item_json.get('answerKey') == choice_label:
                if any_correct:
                    raise ValueError("More than one correct answer found for {item_json}!")
                any_correct = True


        if not any_correct and 'answerKey' in item_json:
            raise ValueError("No correct answer found for {item_json}!")


        answer_id = choice_label_to_id.get(item_json.get("answerKey"))
        # Pad choices with empty strings if not right number
        if len(choice_text_list) != num_choices:
            choice_text_list = (choice_text_list + num_choices * [''])[:num_choices]
            choice_context_list = (choice_context_list + num_choices * [None])[:num_choices]
            if answer_id is not None and answer_id >= num_choices:
                logging.warning(f"Skipping question with more than {num_choices} answers: {item_json}")
                continue

        item_ids.append(item_id)
        questions.append(question_text)
        choice_lists.append(choice_text_list)
        answer_ids.append(answer_id)

    data_file.close()
    return questions, choice_lists, answer_ids

In [ ]:
class BERTDataset(Dataset):  # Only difference is that BERTDataset has token_type_ids while RoBERTaDataset doesn't
    
    def __init__(self, questions, choices, answer_ids, tokenizer):
        out = tokenizer(questions)
        self.input_ids = out["input_ids"]
        self.token_type_ids = out["token_type_ids"]
        self.attention_mask = out["attention_mask"]
        self.questions = questions
        self.choices = choices
        self.answer_ids = answer_ids
        
    def __len__(self):
        return len(self.questions)

    def __getitem__(self, i):
        return {
            "input_ids": self.input_ids[i], 
            "attention_mask": self.attention_mask[i], 
            "token_type_ids": self.token_type_ids[i],
            "choice_list": self.choices[i], 
            "answer_id": self.answer_ids[i],
        }
    

class RoBERTaDataset(Dataset):
    
    def __init__(self, questions, choices, answer_ids, tokenizer):
        if any(prefix in args.model_name_or_path.lower() for prefix in ("roberta", "bart")):
            questions = [question.replace('[MASK]','<mask>') for question in questions]
        out = tokenizer(questions, max_length=45, padding="max_length")
        self.input_ids = out["input_ids"]
        self.attention_mask = out["attention_mask"]
        self.questions = questions
        self.choices = choices
        self.answer_ids = answer_ids
        
    def __len__(self):
        return len(self.questions)

    def __getitem__(self, i):
        return {
            "input_ids": self.input_ids[i], 
            "attention_mask": self.attention_mask[i], 
            "choice_list": self.choices[i], 
            "answer_id": self.answer_ids[i],
        }

In [ ]:
def evaluate_qa_task(args, model, tokenizer, eval_dataset, data_path):
   
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.per_device_eval_batch_size)

    logger.info(f"***** Running evaluation  *****")
    logger.info(f"  Num examples = {len(eval_dataset)}")
    logger.info(f"  Batch size = {args.eval_batch_size}")
    eval_dataloader = tqdm(eval_dataloader, desc="Evaluating")

    all_answers = []
    all_preds = []

    MASK_ID = tokenizer.encode(tokenizer.mask_token)
    MASK_ID = MASK_ID[0]

    for batch in eval_dataloader:
        model.eval()
        
        for i in range(len(batch["choice_list"][0])):
            all_answers.append(batch["choice_list"][batch["answer_id"][i]][i])

     
        choice_lists = batch.pop("choice_list")

        del batch["answer_id"] 
        for key in batch:
            batch[key] = torch.stack(batch[key], dim=-1).cuda()
      
        
        with torch.no_grad():
          if data_path == "hypernym_conjunction_dev.jsonl":
            #replace [MASK] with the index of first pad token as it will be the last token 
            for i in range(len(batch["input_ids"])):
                  question = batch["input_ids"][i]
                  MASK_INDEX = (question==tokenizer.mask_token_id).nonzero().item()
                  batch["input_ids"][i, MASK_INDEX] = 220
            
          outputs = model(**batch)
          logits = outputs.logits
          logits = torch.nn.functional.softmax(logits, dim=2)
          choice_ids = []

          for i, logit in enumerate(logits):  
                first_pad_index = batch["input_ids"][i].tolist().index(tokenizer.eos_token_id)
                x =[" " + choice_lists[j][i] for j in range(len(choice_lists))]
                choice_ids = torch.tensor([tokenizer.encode(" " + choice_lists[j][i], add_special_tokens=False)[0] for j in range(len(choice_lists))])
                choice_ids = choice_ids.cuda()
                probs = logit[first_pad_index-1].index_select(0, choice_ids).cuda()
                max_ind = torch.argmax(probs)
                all_preds.append(choice_lists[max_ind][i])
 
    return all_answers, all_preds
        

In [ ]:
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/conjunction/conjunction_filt4_dev.jsonl.gz
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/hypernym_conjunction/hypernym_conjunction_dev.jsonl.gz
!wget https://olmpics.s3.us-east-2.amazonaws.com/challenge/composition/composition_v2_dev.jsonl.gz

--2021-10-10 20:17:28--  https://olmpics.s3.us-east-2.amazonaws.com/challenge/conjunction/conjunction_filt4_dev.jsonl.gz
Resolving olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)... 52.219.97.170
Connecting to olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)|52.219.97.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26080 (25K) [binary/octet-stream]
Saving to: ‘conjunction_filt4_dev.jsonl.gz’

conjunction_filt4_d 100%[===================>]  25.47K   138KB/s    in 0.2s    

2021-10-10 20:17:29 (138 KB/s) - ‘conjunction_filt4_dev.jsonl.gz’ saved [26080/26080]

--2021-10-10 20:17:29--  https://olmpics.s3.us-east-2.amazonaws.com/challenge/hypernym_conjunction/hypernym_conjunction_dev.jsonl.gz
Resolving olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)... 52.219.97.170
Connecting to olmpics.s3.us-east-2.amazonaws.com (olmpics.s3.us-east-2.amazonaws.com)|52.219.97.170|:443... connected.
HTTP req

In [ ]:
!gunzip conjunction_filt4_dev.jsonl.gz
!gunzip hypernym_conjunction_dev.jsonl.gz
!gunzip composition_v2_dev.jsonl.gz

10/10/2021 20:48:59: Reading QA instances from jsonl dataset at: composition_v2_dev.jsonl


  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
model = transformers.AutoModelWithLMHead.from_pretrained(args.model_name_or_path).cuda()
tokenizer = transformers.AutoTokenizer.from_pretrained(args.model_name_or_path , mask_token = '[MASK]')
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:664: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
10/10/2021 20:38:13: Attempting to acquire lock 139650166699472 on /root/.cache/huggingface/transformers/d82fb41558a2cc40bb6e10a57bbfbd9ff2f3c6614072f05afdfa8f44d566d2ba.55d263c4ba1f8b022997c21dfa03fb8933c57bc9c978354e0a62896cfd837a89.lock
10/10/2021 20:38:13: Lock 139650166699472 acquired on /root/.cache/huggingface/transformers/d82fb41558a2cc40bb6e10a57bbfbd9ff2f3c6614072f05afdfa8f44d566d2ba.55d263c4ba1f8b022997c21dfa03fb8933c57bc9c978354e0a62896cfd837a89.lock


Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

10/10/2021 20:38:14: Attempting to release lock 139650166699472 on /root/.cache/huggingface/transformers/d82fb41558a2cc40bb6e10a57bbfbd9ff2f3c6614072f05afdfa8f44d566d2ba.55d263c4ba1f8b022997c21dfa03fb8933c57bc9c978354e0a62896cfd837a89.lock
10/10/2021 20:38:14: Lock 139650166699472 released on /root/.cache/huggingface/transformers/d82fb41558a2cc40bb6e10a57bbfbd9ff2f3c6614072f05afdfa8f44d566d2ba.55d263c4ba1f8b022997c21dfa03fb8933c57bc9c978354e0a62896cfd837a89.lock
10/10/2021 20:38:14: Attempting to acquire lock 139650177315088 on /root/.cache/huggingface/transformers/234578a5793e64713ba846b4c5e181e043f48b33140622e2c1dd623b665de3f9.4780ef91b17260f8dac8a3c2183aa338b27365326fb706e74db40b03749f8aba.lock
10/10/2021 20:38:14: Lock 139650177315088 acquired on /root/.cache/huggingface/transformers/234578a5793e64713ba846b4c5e181e043f48b33140622e2c1dd623b665de3f9.4780ef91b17260f8dac8a3c2183aa338b27365326fb706e74db40b03749f8aba.lock


Downloading:   0%|          | 0.00/3.02G [00:00<?, ?B/s]

10/10/2021 20:39:54: Attempting to release lock 139650177315088 on /root/.cache/huggingface/transformers/234578a5793e64713ba846b4c5e181e043f48b33140622e2c1dd623b665de3f9.4780ef91b17260f8dac8a3c2183aa338b27365326fb706e74db40b03749f8aba.lock
10/10/2021 20:39:54: Lock 139650177315088 released on /root/.cache/huggingface/transformers/234578a5793e64713ba846b4c5e181e043f48b33140622e2c1dd623b665de3f9.4780ef91b17260f8dac8a3c2183aa338b27365326fb706e74db40b03749f8aba.lock
10/10/2021 20:40:12: Attempting to acquire lock 139650215559056 on /root/.cache/huggingface/transformers/79f5e05af067df502528a0d902e82c24c3f1df9ae570c91fcc38e1f3c0af4c45.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f.lock
10/10/2021 20:40:12: Lock 139650215559056 acquired on /root/.cache/huggingface/transformers/79f5e05af067df502528a0d902e82c24c3f1df9ae570c91fcc38e1f3c0af4c45.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f.lock


...

10/10/2021 20:40:13: Attempting to release lock 139650215559056 on /root/.cache/huggingface/transformers/79f5e05af067df502528a0d902e82c24c3f1df9ae570c91fcc38e1f3c0af4c45.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f.lock
10/10/2021 20:40:13: Lock 139650215559056 released on /root/.cache/huggingface/transformers/79f5e05af067df502528a0d902e82c24c3f1df9ae570c91fcc38e1f3c0af4c45.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f.lock
10/10/2021 20:40:14: Attempting to acquire lock 139650239824784 on /root/.cache/huggingface/transformers/7f7bf8a7802a708af08a812bfbdec9335f2c30f761ec14a8cd17b0d61c818876.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
10/10/2021 20:40:14: Lock 139650239824784 acquired on /root/.cache/huggingface/transformers/7f7bf8a7802a708af08a812bfbdec9335f2c30f761ec14a8cd17b0d61c818876.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

10/10/2021 20:40:16: Attempting to release lock 139650239824784 on /root/.cache/huggingface/transformers/7f7bf8a7802a708af08a812bfbdec9335f2c30f761ec14a8cd17b0d61c818876.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
10/10/2021 20:40:16: Lock 139650239824784 released on /root/.cache/huggingface/transformers/7f7bf8a7802a708af08a812bfbdec9335f2c30f761ec14a8cd17b0d61c818876.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
10/10/2021 20:40:17: Attempting to acquire lock 139650240412752 on /root/.cache/huggingface/transformers/f1179e28982928f50ca02b0188fcd80fb4fa871ba1719df5bf81ac308d0d10af.cf2d0ecb83b6df91b3dbb53f1d1e4c311578bfd3aa0e04934215a49bf9898df0.lock
10/10/2021 20:40:17: Lock 139650240412752 acquired on /root/.cache/huggingface/transformers/f1179e28982928f50ca02b0188fcd80fb4fa871ba1719df5bf81ac308d0d10af.cf2d0ecb83b6df91b3dbb53f1d1e4c311578bfd3aa0e04934215a49bf9898df0.lock


Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

10/10/2021 20:40:19: Attempting to release lock 139650240412752 on /root/.cache/huggingface/transformers/f1179e28982928f50ca02b0188fcd80fb4fa871ba1719df5bf81ac308d0d10af.cf2d0ecb83b6df91b3dbb53f1d1e4c311578bfd3aa0e04934215a49bf9898df0.lock
10/10/2021 20:40:19: Lock 139650240412752 released on /root/.cache/huggingface/transformers/f1179e28982928f50ca02b0188fcd80fb4fa871ba1719df5bf81ac308d0d10af.cf2d0ecb83b6df91b3dbb53f1d1e4c311578bfd3aa0e04934215a49bf9898df0.lock
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
args = get_args()
transformers.set_seed(args.seed)

'''
data = "hypernym_conjunction_dev.jsonl", args.num_choices = 3
data = "composition_v2_dev.jsonl", args.num_choices = 3
data = "conjunction_filt4_dev.jsonl", args.num_choices = 3
'''
args.num_choices = 3
args.model_name_or_path = 'gpt2'
data_path = "conjunction_filt4_dev.jsonl"
data = data_path


#train_questions, train_choices, train_answer_ids = get_data(, args.sample_train, args.num_choices)
eval_questions, eval_choices, eval_answer_ids = get_data(data_path, args.sample_eval, args.num_choices)

10/10/2021 20:52:56: Reading QA instances from jsonl dataset at: conjunction_filt4_dev.jsonl


  0%|          | 0/483 [00:00<?, ?it/s]

In [ ]:
AgeDataset = RoBERTaDataset if any(prefix in args.model_name_or_path.lower() for prefix in ("roberta", "bart", "distil", "gpt")) else BERTDataset

In [ ]:
eval_questions, eval_choices, eval_answer_ids = get_data(data_path, args.sample_eval, args.num_choices)

10/10/2021 20:52:59: Reading QA instances from jsonl dataset at: conjunction_filt4_dev.jsonl


  0%|          | 0/483 [00:00<?, ?it/s]

In [ ]:
accuracy = []
for i in range(5):
  eval_questions, eval_choices, eval_answer_ids = get_data(data, args.sample_eval, args.num_choices)
  combined_dataset = {'que': eval_questions, 'choices': eval_choices, 'ids': eval_answer_ids, }
  combined_dataset = pd.DataFrame(data=combined_dataset)
  sampled_dataset = combined_dataset.sample(frac = 0.8)
  eval_questions = list(sampled_dataset['que'])
  eval_choices = list(sampled_dataset['choices'])
  eval_answer_ids = list(sampled_dataset['ids'])

  eval_dataset = AgeDataset(eval_questions, eval_choices, eval_answer_ids, tokenizer)
  all_answers, all_preds = evaluate_qa_task(args, model, tokenizer, eval_dataset, data_path)
  a = 0
  b = 0
  for i in range(len(all_answers)):
    if all_preds[i] != -1:
        b += 1
        if all_preds[i] == all_answers[i]:
            a += 1
  current_acc = a/b
  accuracy.append(current_acc)

10/10/2021 20:52:59: Reading QA instances from jsonl dataset at: conjunction_filt4_dev.jsonl


  0%|          | 0/483 [00:00<?, ?it/s]

10/10/2021 20:52:59: ***** Running evaluation  *****
10/10/2021 20:52:59:   Num examples = 386
10/10/2021 20:52:59:   Batch size = 2


Evaluating:   0%|          | 0/193 [00:00<?, ?it/s]

10/10/2021 20:53:34: Reading QA instances from jsonl dataset at: conjunction_filt4_dev.jsonl


...

Evaluating:   0%|          | 0/193 [00:00<?, ?it/s]

10/10/2021 20:55:17: Reading QA instances from jsonl dataset at: conjunction_filt4_dev.jsonl


  0%|          | 0/483 [00:00<?, ?it/s]

10/10/2021 20:55:17: ***** Running evaluation  *****
10/10/2021 20:55:17:   Num examples = 386
10/10/2021 20:55:17:   Batch size = 2


Evaluating:   0%|          | 0/193 [00:00<?, ?it/s]

In [ ]:
accuracy

[0.45595854922279794,
 0.47668393782383417,
 0.4533678756476684,
 0.4792746113989637,
 0.4689119170984456]

In [ ]:
#create 95% confidence interval for population mean weight
mini, maxi = st.t.interval(alpha=0.95, df=len(accuracy)-1, loc=np.mean(accuracy), scale=st.sem(accuracy))
accuracy = np.array(accuracy)

In [ ]:
print("The accuracy is of {} for {} task is {} +- {}".format(args.model_name_or_path, data_path, accuracy.mean()*100 ,-1*accuracy.mean()*100+maxi*100))

The accuracy is of gpt2-large for conjunction_filt4_dev.jsonl task is 46.68393782383419 +- 1.4635311049340203


In [ ]:
#gpt2
#hypernym_conjunction_dev - taxonomy
#[0.47807933, 0.47807933, 0.49060543, 0.49478079, 0.47807933]
# 48.39248434237996 +- 1.0106257733482593

#composition_v2_dev - encyclopedia
#[0.3425, 0.315, 0.3175, 0.3175, 0.3025]
#31.900000000000002 +- 1.8046893183785677

#conjunction_filt4_dev - propoerty
#[0.422279792746114,  0.4326424870466321,  0.43005181347150256,  0.43523316062176165,  0.4326424870466321]
#43.056994818652846 +- 0.6187534684196834

In [ ]:
#gpt2-medium
#hypernym_conjunction_dev
#[0.4822546972860125,
 0.4906054279749478,
 0.4801670146137787,
 0.5135699373695198,
 0.48851774530271397]
# 49.102296450939455 +- 1.6537344753589593

#composition_v2_dev
#[0.3425, 0.315, 0.3175, 0.3175, 0.3025]
# 31.900000000000002 +- 1.8046893183785677

#conjunction_filt4_dev
#[0.422279792746114,
 0.4326424870466321,
 0.43005181347150256,
 0.43523316062176165,
 0.4326424870466321]
# 43.056994818652846 +- 0.6187534684196834

In [ ]:
#gpt2-large
#hypernym_conjunction_dev
#[0.4592901878914405,
 0.4613778705636743,
 0.4613778705636743,
 0.4822546972860125,
 0.4822546972860125]
# 46.93110647181628 +- 1.470945261131689

#composition_v2_dev
#[0.375, 0.36, 0.35, 0.345, 0.3475]
#46.93110647181628 +- 1.47094526113168

#conjunction_filt4_dev
#[0.45595854922279794,
 0.47668393782383417,
 0.4533678756476684,
 0.4792746113989637,
 0.4689119170984456]
#46.68393782383419 +- 1.4635311049340203